In [3]:
#import libraries
import pandas as pd
import os
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from scipy import stats
import scipy 
import numpy as np
from tqdm.notebook import tqdm
from torch import nn, optim
import time  
import torch 
import random
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datetime import date
import pickle
import sys
from pathlib import Path  
import re


In [4]:
################################################################################################
folder_name = r'C:\Users\Rania\Desktop\GED issue' 
################################################################################################
os.chdir(folder_name)
cwd = os.getcwd()
################################################################################################
file_name = r'RP9534 V2 BF T1-5.xlsx' 
################################################################################################

In [10]:
new_table = pd.read_excel(file_name)

In [11]:
c_names = ['Right Array', 'Left Array', 'Gait CycleR', 'Gait CycleL']

for c in c_names:
 
    new_table.insert(loc=2,column=c, value = -1)

In [12]:
new_table

,Time,Trials,Gait CycleL,Gait CycleR,Left Array,Right Array,Acceleration Ankles Dorsiflexion-Plantarflexion Right,Acceleration Ankles Dorsiflexion-Plantarflexion Left,Acceleration Ankles Eversion-Inversion Right,Acceleration Ankles Eversion-Inversion Left,...,Velocity Hips Rotation Left,Velocity Knees Abduction-Adduction Right,Velocity Knees Abduction-Adduction Left,Velocity Knees Flexion-Extension Right,Velocity Knees Flexion-Extension Left,Velocity Knees Rotation Right,Velocity Knees Rotation Left,Velocity Pelvis - Pelvis Tilt,Velocity Pelvis - Pelvis Obliquity,Velocity Pelvis - Pelvis Rotation
0,0.00,0,-1,-1,-1,-1,-0.04111,0.27584,-0.17577,0.06732,...,0.01173,-0.04573,-0.00067,-0.00415,0.01421,0.03929,-0.06561,-0.00362,-0.02518,0.03490
1,0.01,0,-1,-1,-1,-1,-0.04111,0.27584,-0.17577,0.06732,...,0.01173,-0.04573,-0.00067,-0.00415,0.01421,0.03929,-0.06561,-0.00362,-0.02518,0.03490
2,0.02,0,-1,-1,-1,-1,-0.03528,0.30336,-0.23879,0.07975,...,0.02022,-0.04966,-0.00296,-0.00300,0.01945,0.02556,-0.07477,-0.00229,-0.02794,0.03567
3,0.03,0,-1,-1,-1,-1,-0.02434,0.31699,-0.30185,0.08938,...,0.02871,-0.05176,-0.00520,-0.00110,0.02437,0.09003,-0.09460,-0.00114,-0.02966,0.03500
4,0.04,0,-1,-1,-1,-1,-0.00798,0.31360,-0.35964,0.09477,...,0.03605,-0.05176,-0.00720,0.00160,0.02851,0.06828,-0.05074,-0.00038,-0.03004,0.03281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43427,434.27,0,-1,-1,-1,-1,23.07487,-10.39467,10.52792,157.91568,...,-4.50323,-4.65553,1.80345,-17.77055,16.02364,-28.27835,-29.39034,-1.81921,-0.72929,-3.54004
43428,434.28,0,-1,-1,-1,-1,13.49802,-11.74450,4.66937,81.03394,...,-2.38669,-2.44924,1.15128,-9.40177,7.74899,-21.71783,-8.35991,-1.03958,-0.38857,-1.84937
43429,434.29,0,-1,-1,-1,-1,5.95441,-6.87275,1.61487,32.40685,...,-0.97792,-1.00539,0.54455,-3.87623,2.93980,-17.07191,10.27031,-0.45426,-0.15991,-0.75073
43430,434.30,0,-1,-1,-1,-1,1.72706,-2.32868,0.38010,8.74419,...,-0.26853,-0.27790,0.16670,-1.07491,0.75836,-13.67493,25.43449,-0.13144,-0.04405,-0.20752


In [14]:
trials = new_table['Trials'].unique()
trials

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [15]:
################################################################################################
GED_folder = r'C:\Users\Rania\Desktop\GED issue\GED' 
################################################################################################

GED_files = sorted(os.listdir(GED_folder))
print("Current working directory has been changed ...")

# Changes the working directory to get the data from their location 
os.chdir(GED_folder)
task2_folder = os.getcwd()
print(f'Current working directory is: {cwd}')
print(f"There are {len(GED_files)} files in the specified path.")


Current working directory has been changed ...
Current working directory is: C:\Users\Rania\Desktop\GED issue
There are 10 files in the specified path.


In [16]:
for t in GED_files:
    print(t)

T1.0 RP9534 V2 BF T1-5.xlsx
T10.0 RP9534 V2 BF T1-5.xlsx
T2.0 RP9534 V2 BF T1-5.xlsx
T3.0 RP9534 V2 BF T1-5.xlsx
T4.0 RP9534 V2 BF T1-5.xlsx
T5.0 RP9534 V2 BF T1-5.xlsx
T6.0 RP9534 V2 BF T1-5.xlsx
T7.0 RP9534 V2 BF T1-5.xlsx
T8.0 RP9534 V2 BF T1-5.xlsx
T9.0 RP9534 V2 BF T1-5.xlsx


In [17]:
sorted_list = sorted(GED_files, key=lambda s: int(re.search(r'\d+', s).group()))
sorted_list

['T1.0 RP9534 V2 BF T1-5.xlsx',
 'T2.0 RP9534 V2 BF T1-5.xlsx',
 'T3.0 RP9534 V2 BF T1-5.xlsx',
 'T4.0 RP9534 V2 BF T1-5.xlsx',
 'T5.0 RP9534 V2 BF T1-5.xlsx',
 'T6.0 RP9534 V2 BF T1-5.xlsx',
 'T7.0 RP9534 V2 BF T1-5.xlsx',
 'T8.0 RP9534 V2 BF T1-5.xlsx',
 'T9.0 RP9534 V2 BF T1-5.xlsx',
 'T10.0 RP9534 V2 BF T1-5.xlsx']

In [21]:
for i, file in enumerate(sorted_list):
    print(i, file)
    if i == 6:
        t_numb = i+1 #trial number 
        continue
    
    t_numb = i+1 #trial number 
    GED_trial_data = pd.read_excel(file)
    
    loc = new_table['Trials'] == t_numb
    index = new_table.index[loc]
    list_ind = index.tolist()
    
    print(len(new_table.loc[list_ind[:-1], 'Left Array']))
    print(len(GED_trial_data['L_stateArray']))
    if len(new_table.loc[list_ind[:-1], 'Left Array']) == len(GED_trial_data['L_stateArray']):
        

        new_table.loc[list_ind[:-1], 'Left Array'] = GED_trial_data['L_stateArray'].values
        new_table.loc[list_ind[:-1], 'Right Array'] = GED_trial_data['R_stateArray'].values
    else:
        print(f'Trial: {t_numb}')
        print(f"original table: {len(new_table.loc[list_ind[:-1], 'Left Array'])}, {len(GED_trial_data['L_stateArray'])}")
        raise Exception('length mismatch between original table and GED table') 


0 T1.0 RP9534 V2 BF T1-5.xlsx
841


KeyError: 'L_stateArray'

In [ ]:
################################################################################################
out_folder_name = r'D:\GED\Task 3 export' 
################################################################################################
out_fname = out_folder_name +  '/' + sorted_list[0].replace("T1 ", "")
new_table.to_excel(out_fname, index=False)  #export data to csv
print(f'exported {out_fname} successfully')